In [1]:
import pandas as pd
import sqlite3

In [3]:
# Conectar a la base de datos SQLite 
conexion = sqlite3.connect('../data/raw/sanoyfresco.db')

In [4]:
# Cargar una tabla completa en un DataFrame
df_sano_fresco = pd.read_sql_query('SELECT * FROM tickets', conexion)
df_sano_fresco

,id_pedido,id_cliente,fecha,hora,id_departamento,id_seccion,id_producto,nombre_producto,precio_unitario,cantidad,precio_total
0,1,112108,2023-02-02 00:00:00,10,4,83,49683,Pepino Kirby,0.99,4,3.96
1,1,112108,2023-02-02 00:00:00,10,4,24,13176,Bolsa de Bananas Orgánicas,2.45,4,9.80
2,1,112108,2023-02-02 00:00:00,10,4,24,47209,Aguacate Hass Orgánico,1.79,2,3.58
3,2,202279,2023-02-03 00:00:00,9,4,83,28985,Col Rizada Orgánica de Michigan,2.55,2,5.10
4,2,202279,2023-02-03 00:00:00,9,4,83,17794,Zanahorias,0.95,5,4.75
...,...,...,...,...,...,...,...,...,...,...,...
4975713,3421080,52726,2023-01-09 00:00:00,11,16,84,27845,Leche Entera Orgánica,0.99,3,2.97
4975714,3421080,52726,2023-01-09 00:00:00,11,4,83,41950,Racimo de Tomates Orgánicos,2.25,7,15.75
4975715,3421080,52726,2023-01-09 00:00:00,11,4,16,31717,Cilantro Orgánico,0.69,5,3.45
4975716,3421082,175185,2023-08-22 00:00:00,18,4,24,16797,Fresas,2.05,4,8.20


In [5]:
# Cerrar conexión con la base de datos
conexion.close()

In [6]:
df_sano_fresco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4975718 entries, 0 to 4975717
Data columns (total 11 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id_pedido        int64  
 1   id_cliente       int64  
 2   fecha            object 
 3   hora             int64  
 4   id_departamento  int64  
 5   id_seccion       int64  
 6   id_producto      int64  
 7   nombre_producto  object 
 8   precio_unitario  float64
 9   cantidad         int64  
 10  precio_total     float64
dtypes: float64(2), int64(7), object(2)
memory usage: 417.6+ MB


In [7]:
# Transformar el tipo de la fecha de object a datatime
df_sano_fresco['fecha'] = pd.to_datetime(df_sano_fresco['fecha'])
df_sano_fresco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4975718 entries, 0 to 4975717
Data columns (total 11 columns):
 #   Column           Dtype         
---  ------           -----         
 0   id_pedido        int64         
 1   id_cliente       int64         
 2   fecha            datetime64[ns]
 3   hora             int64         
 4   id_departamento  int64         
 5   id_seccion       int64         
 6   id_producto      int64         
 7   nombre_producto  object        
 8   precio_unitario  float64       
 9   cantidad         int64         
 10  precio_total     float64       
dtypes: datetime64[ns](1), float64(2), int64(7), object(1)
memory usage: 417.6+ MB


In [8]:
# Crear un mini DataFrame
df_cesta = df_sano_fresco[['id_pedido','nombre_producto']]
df_cesta

,id_pedido,nombre_producto
0,1,Pepino Kirby
1,1,Bolsa de Bananas Orgánicas
2,1,Aguacate Hass Orgánico
3,2,Col Rizada Orgánica de Michigan
4,2,Zanahorias
...,...,...
4975713,3421080,Leche Entera Orgánica
4975714,3421080,Racimo de Tomates Orgánicos
4975715,3421080,Cilantro Orgánico
4975716,3421082,Fresas


In [10]:
# Agrupar los productos por pedido
df_agrupado = df_cesta.groupby('id_pedido')['nombre_producto'].apply(lambda producto: ','.join(producto))
df_agrupado

id_pedido
1          Pepino Kirby,Bolsa de Bananas Orgánicas,Aguaca...
2                 Col Rizada Orgánica de Michigan,Zanahorias
3                                   Espinacas Baby Orgánicas
5          Bolsa de Bananas Orgánicas,Frambuesas Orgánica...
10         Banana,Cilantro Orgánico,Aguacate Orgánico,Ceb...
                                 ...                        
3421077    Frambuesas Orgánicas,Calabacín Orgánico,Leche ...
3421078                       Manzanas Gala Orgánicas,Banana
3421080    Leche Entera Orgánica,Racimo de Tomates Orgáni...
3421082                                               Fresas
3421083                                               Banana
Name: nombre_producto, Length: 2060188, dtype: object

In [11]:
# Aplicar pd.get_dummies() para transformart los productos en columnas con 0/1
df_transcciones = df_agrupado.str.get_dummies(sep=',')
df_transcciones

,Agua con Gas de Pomelo,Aguacate Hass Orgánico,Aguacate Orgánico,Ajo Orgánico,Apio Orgánico en Ramillete Pequeño,Arándanos Orgánicos,Banana,Bolsa de Bananas Orgánicas,Calabacín Orgánico,Cebolla Amarilla Orgánica,...,Manzana Honeycrisp Orgánica,Manzanas Gala Orgánicas,Pepino Kirby,Pepino Orgánico,Racimo de Tomates Orgánicos,Rúcula Baby Orgánica,Tomates Cherry Orgánicos,Uvas Rojas sin Semillas,Zanahorias,Zanahorias Baby Orgánicas
id_pedido,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421077,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3421078,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3421080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [13]:
# Soporte para cada producto
soporte =  df_transcciones.mean() * 100
soporte.sort_values(ascending=False)

Banana                                23.809381
Bolsa de Bananas Orgánicas            19.129710
Fresas Orgánicas                      13.349170
Espinacas Baby Orgánicas              12.198935
Aguacate Hass Orgánico                10.703004
Aguacate Orgánico                      8.929331
Limón Grande                           7.791279
Fresas                                 7.241960
Limones                                7.107944
Leche Entera Orgánica                  6.918301
Frambuesas Orgánicas                   6.906797
Cebolla Amarilla Orgánica              5.706324
Ajo Orgánico                           5.521778
Calabacín Orgánico                     5.302429
Arándanos Orgánicos                    5.086138
Pepino Kirby                           4.840044
Manzana Fuji Orgánica                  4.502841
Limón Orgánico                         4.422266
Tomates Cherry Orgánicos               4.266795
Manzana Honeycrisp Orgánica            4.231216
Uvas Rojas sin Semillas                4

In [ ]:
# Función para calcular la confianza entre dos productos en la muestra
def confianza(antecedente, consecuente):
    # Casos donde se comparon ambos productos
    conjunto_ac = df_transcciones[(df_transcciones[antecedente] == 1) & 
                                  (df_transcciones[consecuente] == 1)]
    # Confianza = compras conjuntas / compras de producto A
    return len(conjunto_ac) / df_transcciones[antecedente].sum()

In [15]:
# Función para calcualr el lift entre dos productos en la muestra
def lift(antecedente, consecuente):
    soporte_a = df_transcciones[antecedente].mean()
    soporte_b = df_transcciones[consecuente].mean()
    conteo_ac = len(df_transcciones[(df_transcciones[antecedente] == 1) &
                                    (df_transcciones[consecuente] == 1)])
    soporte_ac = conteo_ac / len(df_transcciones)
    return soporte_ac / (soporte_a * soporte_b)

In [ ]:
from itertools import combinations

# Definir un umbral para la confianza mínima
